In [108]:
%matplotlib inline
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [109]:
movies_train_df = pd.read_csv(Path('movies.csv'))

In [110]:
movies_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7930 entries, 0 to 7929
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      7930 non-null   object 
 1   rating    7853 non-null   object 
 2   genre     7930 non-null   object 
 3   year      7930 non-null   int64  
 4   released  7928 non-null   object 
 5   score     7927 non-null   float64
 6   votes     7665 non-null   float64
 7   director  7930 non-null   object 
 8   writer    7927 non-null   object 
 9   star      7929 non-null   object 
 10  country   7665 non-null   object 
 11  budget    5497 non-null   float64
 12  gross     7741 non-null   object 
 13  company   7913 non-null   object 
 14  runtime   7926 non-null   float64
dtypes: float64(4), int64(1), object(10)
memory usage: 929.4+ KB


In [111]:
movies_train_df = movies_train_df.drop('budget', axis = 1)

In [112]:
movies_train_df = movies_train_df.dropna(axis = 0, how='any')
movies_train_df

,name,rating,genre,year,released,score,votes,director,writer,star,country,gross,company,runtime
0,The Shining,R,Drama,1980,13-Jun-80,8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,46998772,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,2-Jul-80,5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,58853106,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,20-Jun-80,8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,538375067,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,2-Jul-80,7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,83453539,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,25-Jul-80,7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,39846344,Orion Pictures,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7651,The Call of the Wild,PG,Adventure,2020,21-Feb-20,6.8,42000.0,Chris Sanders,Michael Green,Harrison Ford,Canada,111105497,20th Century Studios,100.0
7652,The Eight Hundred,Not Rated,Action,2020,28-Aug-20,6.8,3700.0,Hu Guan,Hu Guan,Zhi-zhong Huang,China,461421559,Beijing Diqi Yinxiang Entertainment,149.0
7653,The Quarry,R,Crime,2020,17-Apr-20,5.4,2400.0,Scott Teems,Scott Teems,Shea Whigham,United States,3661,Prowess Pictures,98.0
7656,Tulsa,PG-13,Comedy,2020,3-Jun-20,5.0,294.0,Scott Pryor,Scott Pryor,Scott Pryor,United States,413378,Pryor Entertainment,120.0


In [113]:
movies_test_df = movies_test_df = pd.read_csv(Path('upcoming_df_2022.csv'))
movies_test_df

,movie_id,title,movie_rating,release_date,runtime,genre_name,Director,Writer,Actors,Language,Poster,metascore,imdb_rating
0,0,Studio 666,R,2/25/2022,0,"Comedy, Horror, Music",BJ McDonnell,"Dave Grohl, Jeff Buhler, Rebecca Hughes","Jenna Ortega, Will Forte, Leslie Grossman",English,https://m.media-amazon.com/images/M/MV5BMmU5Nj...,0,0.0
1,1,Butter,0,1/31/2022,19,"Short, Comedy, Drama",Anita Lee,Madeleine Withington,"Amber Adams, Matthew Backer, Jenna Bell",English,0,0,0.0
2,2,The Batman,PG-13,3/4/2022,175,"Action, Crime, Drama",Matt Reeves,"Matt Reeves, Peter Craig, Bob Kane","Barry Keoghan, Zoë Kravitz, Robert Pattinson",English,https://m.media-amazon.com/images/M/MV5BYTExZT...,0,0.0
3,4,He Was Asking for It: A Noir Film,0,4/30/2022,0,"Short, Crime",Rocky Karlage,"Rocky Karlage, Staci Layne Wilson","Rocky Karlage, Patrick Lamba, Charlie Jiin",English,https://m.media-amazon.com/images/M/MV5BMjA5NG...,0,0.0
4,5,X,R,3/18/2022,0,Horror,Ti West,Ti West,"Jenna Ortega, Mia Goth, Brittany Snow",English,https://m.media-amazon.com/images/M/MV5BYjI1OW...,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,75,Strange World,0,11/23/2022,0,"Animation, Action, Adventure",Don Hall,Qui Nguyen,Alan Tudyk,English,https://m.media-amazon.com/images/M/MV5BMDNjOT...,0,0.0
71,76,Avatar 2,0,12/16/2022,0,"Action, Adventure, Sci-Fi",James Cameron,"James Cameron, Josh Friedman","Kate Winslet, Zoe Saldana, Michelle Yeoh",English,https://m.media-amazon.com/images/M/MV5BNmM1Nm...,0,0.0
72,77,Aquaman and the Lost Kingdom,0,12/16/2022,0,"Action, Adventure, Fantasy",James Wan,"David Leslie Johnson-McGoldrick, Paul Norris, ...","Temuera Morrison, Jason Momoa, Nicole Kidman",English,https://m.media-amazon.com/images/M/MV5BM2QyYT...,0,0.0
73,78,Untitled Illumination Entertainment Project,0,12/21/2022,0,"Animation, Adventure, Comedy","Aaron Horvath, Michael Jelenic","Matthew Fogel, Shigeru Miyamoto","Anya Taylor-Joy, Chris Pratt, Seth Rogen",English,https://m.media-amazon.com/images/M/MV5BODMzNm...,0,0.0


In [114]:
movies_test_df = movies_test_df.dropna(axis = 0, how='any')
movies_test_df

,movie_id,title,movie_rating,release_date,runtime,genre_name,Director,Writer,Actors,Language,Poster,metascore,imdb_rating
0,0,Studio 666,R,2/25/2022,0,"Comedy, Horror, Music",BJ McDonnell,"Dave Grohl, Jeff Buhler, Rebecca Hughes","Jenna Ortega, Will Forte, Leslie Grossman",English,https://m.media-amazon.com/images/M/MV5BMmU5Nj...,0,0.0
1,1,Butter,0,1/31/2022,19,"Short, Comedy, Drama",Anita Lee,Madeleine Withington,"Amber Adams, Matthew Backer, Jenna Bell",English,0,0,0.0
2,2,The Batman,PG-13,3/4/2022,175,"Action, Crime, Drama",Matt Reeves,"Matt Reeves, Peter Craig, Bob Kane","Barry Keoghan, Zoë Kravitz, Robert Pattinson",English,https://m.media-amazon.com/images/M/MV5BYTExZT...,0,0.0
3,4,He Was Asking for It: A Noir Film,0,4/30/2022,0,"Short, Crime",Rocky Karlage,"Rocky Karlage, Staci Layne Wilson","Rocky Karlage, Patrick Lamba, Charlie Jiin",English,https://m.media-amazon.com/images/M/MV5BMjA5NG...,0,0.0
4,5,X,R,3/18/2022,0,Horror,Ti West,Ti West,"Jenna Ortega, Mia Goth, Brittany Snow",English,https://m.media-amazon.com/images/M/MV5BYjI1OW...,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,75,Strange World,0,11/23/2022,0,"Animation, Action, Adventure",Don Hall,Qui Nguyen,Alan Tudyk,English,https://m.media-amazon.com/images/M/MV5BMDNjOT...,0,0.0
71,76,Avatar 2,0,12/16/2022,0,"Action, Adventure, Sci-Fi",James Cameron,"James Cameron, Josh Friedman","Kate Winslet, Zoe Saldana, Michelle Yeoh",English,https://m.media-amazon.com/images/M/MV5BNmM1Nm...,0,0.0
72,77,Aquaman and the Lost Kingdom,0,12/16/2022,0,"Action, Adventure, Fantasy",James Wan,"David Leslie Johnson-McGoldrick, Paul Norris, ...","Temuera Morrison, Jason Momoa, Nicole Kidman",English,https://m.media-amazon.com/images/M/MV5BM2QyYT...,0,0.0
73,78,Untitled Illumination Entertainment Project,0,12/21/2022,0,"Animation, Adventure, Comedy","Aaron Horvath, Michael Jelenic","Matthew Fogel, Shigeru Miyamoto","Anya Taylor-Joy, Chris Pratt, Seth Rogen",English,https://m.media-amazon.com/images/M/MV5BODMzNm...,0,0.0


In [115]:
movies_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7412 entries, 0 to 7659
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      7412 non-null   object 
 1   rating    7412 non-null   object 
 2   genre     7412 non-null   object 
 3   year      7412 non-null   int64  
 4   released  7412 non-null   object 
 5   score     7412 non-null   float64
 6   votes     7412 non-null   float64
 7   director  7412 non-null   object 
 8   writer    7412 non-null   object 
 9   star      7412 non-null   object 
 10  country   7412 non-null   object 
 11  gross     7412 non-null   object 
 12  company   7412 non-null   object 
 13  runtime   7412 non-null   float64
dtypes: float64(3), int64(1), object(10)
memory usage: 868.6+ KB


In [116]:
movies_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 0 to 74
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movie_id      75 non-null     int64  
 1   title         75 non-null     object 
 2   movie_rating  75 non-null     object 
 3   release_date  75 non-null     object 
 4   runtime       75 non-null     int64  
 5   genre_name    75 non-null     object 
 6   Director      75 non-null     object 
 7   Writer        75 non-null     object 
 8   Actors        75 non-null     object 
 9   Language      75 non-null     object 
 10  Poster        75 non-null     object 
 11  metascore     75 non-null     int64  
 12  imdb_rating   75 non-null     float64
dtypes: float64(1), int64(3), object(9)
memory usage: 8.2+ KB


In [117]:
y = movies_train_df['score']
X = movies_train_df.drop('score', axis = 1)
# X.info()

In [118]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7412 entries, 0 to 7659
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      7412 non-null   object 
 1   rating    7412 non-null   object 
 2   genre     7412 non-null   object 
 3   year      7412 non-null   int64  
 4   released  7412 non-null   object 
 5   votes     7412 non-null   float64
 6   director  7412 non-null   object 
 7   writer    7412 non-null   object 
 8   star      7412 non-null   object 
 9   country   7412 non-null   object 
 10  gross     7412 non-null   object 
 11  company   7412 non-null   object 
 12  runtime   7412 non-null   float64
dtypes: float64(2), int64(1), object(10)
memory usage: 810.7+ KB


In [119]:
y

0       8.4
1       5.8
2       8.7
3       7.7
4       7.3
       ... 
7651    6.8
7652    6.8
7653    5.4
7656    5.0
7659    3.4
Name: score, Length: 7412, dtype: float64

In [120]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
transformed = lb_make.fit_transform(X['name'])
X['name'] = transformed

transformed = lb_make.fit_transform(X['rating'])
X['rating'] = transformed

transformed = lb_make.fit_transform(X['genre'])
X['genre'] = transformed

transformed = lb_make.fit_transform(X['director'])
X['director'] = transformed

transformed = lb_make.fit_transform(X['writer'])
X['writer'] = transformed

transformed = lb_make.fit_transform(X['star'])
X['star'] = transformed

transformed = lb_make.fit_transform(X['country'])
X['country'] = transformed

transformed = lb_make.fit_transform(X['company'])
X['company'] = transformed

transformed = lb_make.fit_transform(X['released'])
X['released'] = transformed

X

,name,rating,genre,year,released,votes,director,writer,star,country,gross,company,runtime
0,6372,6,6,1980,434,927000.0,2471,3851,979,52,46998772,2179,146.0
1,5373,6,1,1980,1084,65000.0,2169,1564,306,53,58853106,686,104.0
2,4956,4,0,1980,1182,1200000.0,1063,2464,1642,53,538375067,1446,124.0
3,277,4,4,1980,1084,221000.0,1248,1920,2112,53,83453539,1697,88.0
4,993,6,4,1980,1672,108000.0,1010,499,387,53,39846344,1665,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7651,5432,4,1,2020,1264,42000.0,438,2838,895,6,111105497,17,100.0
7652,5586,3,0,2020,1917,3700.0,1048,1602,2645,8,461421559,350,149.0
7653,6265,6,5,2020,765,2400.0,2404,3726,2300,53,3661,1786,98.0
7656,6792,5,4,2020,2119,294.0,2399,3719,2241,53,413378,1788,120.0


In [121]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [122]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled

array([[-0.57018375,  0.69423572,  0.54519736, ..., -0.23782861,
        -0.98818259,  1.08394537],
       [-0.55347165,  0.69423572,  0.54519736, ..., -0.46517883,
         0.65110402, -0.28976709],
       [-1.66363264,  0.69423572,  0.16353861, ..., -0.42528305,
         1.32984746, -0.50956108],
       ...,
       [-0.47086612, -0.17457481,  0.92685612, ..., -0.32091671,
        -0.24277685, -0.12492159],
       [ 0.20860016,  0.69423572,  0.92685612, ..., -0.44779464,
         1.07531867,  1.13889387],
       [-1.47120474, -0.17457481,  0.16353861, ..., -0.06947432,
        -1.32906936, -0.45461258]])

In [123]:
classifier = LogisticRegression()
classifier

LogisticRegression()

In [124]:
# classifier.fit(X_train, y_train)

ValueError: Unknown label type: 'continuous'